In [ ]:
!cp -r ../input/lux-ai-2021/* .
!pip install kaggle-environments -U

In [ ]:
from lux.game import Game
from lux.game_map import Cell, RESOURCE_TYPES, Position
from lux.constants import Constants
from lux.game_constants import GAME_CONSTANTS
from lux import annotate
import math
import random
import numpy as np
from tqdm import tqdm
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import layers
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.compat.v1 import ConfigProto, Session
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow import keras
from kaggle_environments import make
import tensorflow as tf
seed = 562124210

In [ ]:
#Solo se le pasan las acciones del jugador aliado
def traductor_updates(updates,player_id):
    research = [0,0] 
    game_map = []
    cities = {}
            
    #inicializamos el array
    game_map = []
    for i in range(12):
        game_map.append([])
        for e in range(12):
            game_map[i].append([])
            for o in range(21):
                game_map[i][e].append(0)
    #hacemos la traducción
    for i in updates:
        temp = i.split(" ")
        if temp[0] == "rp":
            if int(temp[1]) == player_id:
                research[0] = int(temp[2])
            else:
                research[1] = int(temp[2])
        elif temp[0] == "r":
            if temp[1] == "wood":
                game_map[int(temp[2])][int(temp[3])][0] = int(temp[4])
            elif temp[1] == "coal":
                game_map[int(temp[2])][int(temp[3])][1] = int(temp[4])
            elif temp[1] == "uranium":
                game_map[int(temp[2])][int(temp[3])][2] = int(temp[4])
        #almacenamos las ciudades para obtener su fuel y fuelupkeep cuando aparezcan sus ct
        elif temp[0] == "c":
            cities[temp[2]] = [int(temp[3]),int(temp[4])]
        elif temp[0] == "ct":
            if int(temp[1]) == player_id:
                #tiene ciudad aliada
                game_map[int(temp[3])][int(temp[4])][3] = 1
                #Fuel
                game_map[int(temp[3])][int(temp[4])][4] = cities[temp[2]][0]
                #FuelUpkeep
                game_map[int(temp[3])][int(temp[4])][5] = cities[temp[2]][1]
                #cd
                game_map[int(temp[3])][int(temp[4])][6] = int(temp[5])
            else:
                #tiene ciudad enemiga
                game_map[int(temp[3])][int(temp[4])][7] = 1
                #Fuel
                game_map[int(temp[3])][int(temp[4])][8] = cities[temp[2]][0]
                #FuelUpkeep
                game_map[int(temp[3])][int(temp[4])][9] = cities[temp[2]][1]
                #cd
                game_map[int(temp[3])][int(temp[4])][10] = int(temp[5])
        elif temp[0] == "u":
            if int(temp[2]) == player_id:
                #num unidades aliadas
                game_map[int(temp[4])][int(temp[5])][11] += 1
                #cooldown = 0
                if int(temp[6]) == 0:
                    game_map[int(temp[4])][int(temp[5])][12] += 1
                #wood
                game_map[int(temp[4])][int(temp[5])][13] = int(temp[7])
                #coal
                game_map[int(temp[4])][int(temp[5])][14] = int(temp[8])
                #uranium
                game_map[int(temp[4])][int(temp[5])][15] = int(temp[9])
            else:
                #num unidades enemigas
                game_map[int(temp[4])][int(temp[5])][16] += 1
                #cd 0
                if int(temp[6]) == 0:
                    game_map[int(temp[4])][int(temp[5])][17] += 1
                #wood
                game_map[int(temp[4])][int(temp[5])][18] = int(temp[7])
                #coal
                game_map[int(temp[4])][int(temp[5])][19] = int(temp[8])
                #uranium
                game_map[int(temp[4])][int(temp[5])][20] = int(temp[9])
    return research, np.array(game_map)

In [ ]:
def construct_q_network(state_dim=12,features_update = 21,output_dim=8):
    shape=(state_dim,state_dim,features_update)
    
    
    inputs1 = layers.Input(shape=(3))
    hidden1 = layers.Dense(6,activation='relu',input_shape=(32,3))(inputs1)
    hidden2 = layers.Dense(6,activation='relu')(hidden1)
    hidden3 = layers.Dense(6,activation='linear')(hidden2)
    
    inputs2 = layers.Input(shape=(state_dim,state_dim,features_update))
    conv1 = layers.Conv2D(12, (5, 5), activation='relu', input_shape=(12, 12, 21))(inputs2)
    conv2 = layers.Conv2D(32, (3, 3), activation='relu', input_shape=(12, 12, 21))(conv1)
    conv3 = layers.Conv2D(12, (3, 3), activation='relu', input_shape=(12, 12, 21))(conv2)
    flatten = layers.Flatten()(conv3)
    
    combined = layers.concatenate([hidden3,flatten])
    dense = layers.Dense(1152)(combined)
    output = layers.Reshape((12,12,8))(dense)
    
    model = Model(inputs=[inputs1, inputs2], outputs=output)
    model.compile(optimizer = "adam", loss="mse", metrics = ["accuracy"])
    return model

In [ ]:
q_network_orig = construct_q_network()
q_network_temp = construct_q_network()

In [ ]:
q_network_orig.summary()

In [ ]:
tf.keras.utils.plot_model(
    q_network_orig,
    to_file="model.png",
    show_shapes=1,
    show_dtype=1,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=False,
    dpi=96)

output: 12x12x8
(c,n,s,e,w,bc|bw,research)

In [ ]:
game_state = None
old_reward = None
old_state = None
epsilon = 1.0
epsilon_decay = 0.995
epsilon_final = 0.1
gamma = 0.95
learning_rate = 0.01
old_output = []
old_research = None
old_game_map = None
global_actions = None
memory=[]
def train_q_network(observation, configuration):
    global game_state
    global q_network_orig
    global q_network_temp
    global old_reward
    global old_research
    global old_game_map
    global epsilon
    global epsilon_decay
    global epsilon_final
    global gamma
    global learning_rate
    global old_output
    global global_actions
    ### Do not edit ###
    if observation["step"] == 0:
        game_state = Game()
        game_state._initialize(observation["updates"])
        game_state._update(observation["updates"][2:])
        game_state.id = observation.player
        old_reward = 100000+10000
        old_output = []
        for i in range(12):
            old_output.append([])
            for e in range(12):
                old_output[i].append([0,0,0,0,0,0,0,0])
    else:
        game_state._update(observation["updates"])
    actions = []

    
    player = game_state.players[observation.player]
    opponent = game_state.players[(observation.player + 1) % 2]
    width, height = game_state.map.width, game_state.map.height
    reward = 0
    q_values_units = {}
    actions_units = {}
    actions_cities = []
    new_actions_units = {}
    new_actions_cities = []
    new_q_values_units = {}
    # Se traduce el estado alcanzado
    research, game_map = traductor_updates(observation["updates"],player.team)
    research.append(observation["step"])
    
    # Se obtienen los q valores para el estado alcanzado
    actions_matrix = q_network_temp.predict([np.array([research]),np.array([game_map])])
    
    # Elegimos las acciones a ejecutar y extraemos sus q valores (máximas o aleatorias)
    sample_epsilon = np.random.rand()
    if sample_epsilon > epsilon:
        for unit in player.units:
            if unit.can_act():
                # Se almacena la acción elegida para el estado actual
                actions_units[unit.id] = np.argmax(actions_matrix[0][unit.pos.x][unit.pos.y][:6])
                # Se almacena el q valor de la acción elegida
                q_values_units[unit.id] = max(actions_matrix[0][unit.pos.x][unit.pos.y][:6])
        for city in player.cities:
            for citytile in player.cities[city].citytiles:
                if citytile.can_act():
                    actions_cities.append([citytile.pos,np.argmax(actions_matrix[0][citytile.pos.x][citytile.pos.y][6:]),max(actions_matrix[0][citytile.pos.x][citytile.pos.y][6:])])
    else:
        for unit in player.units:
            if unit.can_act():
                temp = random.randint(0,5)
                actions_units[unit.id] = temp
                q_values_units[unit.id] = actions_matrix[0][unit.pos.x][unit.pos.y][temp]
        for city in player.cities:
            for citytile in player.cities[city].citytiles:
                if citytile.can_act():
                    temp = random.randint(6,7)
                    actions_cities.append([citytile.pos,temp,actions_matrix[0][citytile.pos.x][citytile.pos.y][temp]])
                    
                    
    if q_values_units:
        for action_unit in actions_units.items():
            if action_unit[1] == 0:
                actions.append("m "+action_unit[0]+" c")
            elif action_unit[1] == 1:
                actions.append("m "+action_unit[0]+" n")
            elif action_unit[1] == 2:
                actions.append("m "+action_unit[0]+" s")
            elif action_unit[1] == 3:
                actions.append("m "+action_unit[0]+" e")
            elif action_unit[1] == 4:
                actions.append("m "+action_unit[0]+" w")
            elif action_unit[1] == 5:
                actions.append("bcity "+action_unit[0])
                
    if actions_cities:
        for action_city in actions_cities:
            if action_city[1] == 6:
                actions.append("bw "+str(action_city[0].x)+" "+str(action_city[0].y))
            if action_city[1] == 7:
                actions.append("r "+str(action_city[0].x)+" "+str(action_city[0].y))
    
    ########
                    
    # Se calcula la recompensa obtenida para el estado alcanzado
    for unit in player.units:
        reward += unit.cargo.wood
        reward += unit.cargo.coal*10
        reward += unit.cargo.uranium*40
    for city in player.cities:
        reward += player.cities[city].fuel
    reward += len(player.cities)*100000+len(player.units)*10000
    r = reward-old_reward
                    
    #Se almacena en la memoria
    if observation["step"] != 0:
        memory.append([old_research,old_game_map,global_actions,r,research, game_map,False])
    
    if len(memory)>= 16:
        samples = random.sample(memory,16)
        q_values_samples = []
        for sample in samples:
            # Calculamos los q valores
            q_values_matrix = q_network_temp.predict([np.array([sample[0]]),np.array([sample[1]])])
            new_q_values_matrix = q_network_orig.predict([np.array([sample[4]]),np.array([sample[5]])])
            # Obtenemos las acciones que se ejecutaron
            c0 = 0
            for i in sample[2][0]:
                c1 = 0
                for e in i:
                    c2 = 0
                    for o in e:
                        if o >= 1:
                            if not sample[6]:
                                q_values_matrix[0][c0][c1][c2] = learning_rate*(r + gamma*new_q_values_matrix[0][c0][c1][c2])
                            else:
                                q_values_matrix[0][c0][c1][c2] = r
                        c2 += 1
                    c1 += 1
                c0 += 1
            q_values_samples.append(q_values_matrix[0])
        sample_research = np.array(samples)[:,0]
        sample_game_map = np.array(samples)[:,1]
        converted_r = []
        converted_gm = []
        converted_qv = []
        for i in sample_research:
            converted_r.append(np.array(i))
        for i in sample_game_map:
            converted_gm.append(np.array(i))
        for i in q_values_samples:
            converted_qv.append(np.array(i))

        if observation["step"]%50 == 0:
            q_network_temp.fit([np.array(converted_r),np.array(converted_gm)],np.array(q_values_samples),epochs=1,verbose=1)
        else:
            q_network_temp.fit([np.array(converted_r),np.array(converted_gm)],np.array(q_values_samples),epochs=1,verbose=0)

    # guardamos los valores para el siguiente estado
    old_research = research
    old_game_map = game_map
    old_reward = reward
    global_actions = actions_matrix
    # aplicamos el decay a epsilon
    if epsilon > epsilon_final:
        epsilon*= epsilon_decay

    return actions

In [ ]:
def last_state(win,state):
    global game_state
    global q_network_temp
    global q_network_orig
    global old_research
    global old_game_map
    global epsilon
    global epsilon_decay
    global epsilon_final
    global gamma
    global learning_rate
    global old_output
    global global_actions
    
    actions = []
    player = game_state.players[1]
    opponent = game_state.players[0]
    width, height = game_state.map.width, game_state.map.height
    reward = 0
    
    research, game_map = traductor_updates(state[0]["observation"]["updates"],player.team)
    research.append(game_state.turn+1)
    
    for unit in player.units:
        reward += unit.cargo.wood
        reward += unit.cargo.coal*10
        reward += unit.cargo.uranium*40
    for city in player.cities:
        reward += player.cities[city].fuel
    reward += len(player.cities)*100000+len(player.units)*10000
    r = reward-old_reward
    
    #Se almacena en la memoria
    memory.append([old_research,old_game_map,global_actions,r,research,game_map,True])
    
    if len(memory)>= 16:
        samples = random.sample(memory,16)
        q_values_samples = []
        for sample in samples:
            # Calculamos los q valores
            q_values_matrix = q_network_temp.predict([np.array([sample[0]]),np.array([sample[1]])])
            new_q_values_matrix = q_network_orig.predict([np.array([sample[4]]),np.array([sample[5]])])
            # Obtenemos las acciones que se ejecutaron
            c0 = 0
            for i in sample[2][0]:
                c1 = 0
                for e in i:
                    c2 = 0
                    for o in e:
                        if o >= 1:
                            if not sample[6]:
                                q_values_matrix[0][c0][c1][c2] = learning_rate*(r + gamma*new_q_values_matrix[0][c0][c1][c2])
                            else:
                                q_values_matrix[0][c0][c1][c2] = r
                        c2 += 1
                    c1 += 1
                c0 += 1
            q_values_samples.append(q_values_matrix[0])
        sample_research = np.array(samples)[:,0]
        sample_game_map = np.array(samples)[:,1]
        converted_r = []
        converted_gm = []
        converted_qv = []
        for i in sample_research:
            converted_r.append(np.array(i))
        for i in sample_game_map:
            converted_gm.append(np.array(i))
        for i in q_values_samples:
            converted_qv.append(np.array(i))

        q_network_temp.fit([np.array(converted_r),np.array(converted_gm)],np.array(q_values_samples),epochs=1,verbose=1)

    # aplicamos el decay a epsilon
    if epsilon > epsilon_final:
        epsilon*= epsilon_decay
    return actions

In [ ]:
episodes = 300
learning_rate = 0.99
winrate = 0
episodes_played = 0
epsilon = 1
env = make("lux_ai_2021", debug=True, configuration={"seed": 562124210,"annotations": True, "width":12, "height":12, "loglevel": 0})
for episodes in tqdm(range(episodes)):
    steps = env.run(["simple_agent", train_q_network])
    episodes_played += 1
    if steps[len(steps)-1][0]["reward"] < steps[len(steps)-1][1]["reward"]:
        last_state(True,steps[len(steps)-1])
        winrate += 1
        print("wins")
        print(winrate)
        print(episodes_played)
    else:
        last_state(False,steps[len(steps)-1])
        print(winrate)
        print(episodes_played)
    q_network_orig = tf.keras.models.clone_model(q_network_temp)
    if learning_rate >= 0.01:
        learning_rate*=0.995
q_network_orig.save("deep_q_learning.h5")

In [ ]:
env.render(mode="ipython", width=1200, height=800)